# Processing script notebooks?

A fun experiment (ab)using SDP control system.

## Introduction


Quick reminder of the SDP processing architecture:
1. The sub-system gets a **processing block** either via Tango, the console or another interface.
2. The **processing controller** starts a **processing script** that drives the processing block
2. This **processing script** requests required resources (which will then be scheduled - at some point)
3. Once these resources have been allocated, it can start **execution engines** to actually, well, execute processing

**Great!** Ticks all boxes, all requirements fulfilled. What more could we want?


## Notebook?


**But!** Dynamic deployment requires processing script to have a Docker image
⇒ Not very dynamic

**What if?** We just have an already-started notebook pose as the processing script?



## Setting the stage

Let's import a bunch of modules. Especially an - ahem - modified version of the SDP scripting library

(The processing scripting library is what processing scripts use to interact with the SDP architecture)

In [1]:
!pip uninstall ska-sdp-scripting -yq
!pip install -U git+https://gitlab.com/ska-telescope/sdp/ska-sdp-scripting@sp-1667-notebook-script -q
import ska_sdp_config, datetime, os, dask, logging, sys, time, ska_sdp_scripting

Let's continue doing our not-at-all-suspect work on a namespace of the DP testing platform:

In [2]:
NAMESPACE = "dp-dppt-peter"  # add the namespace you want to connect to here
cfg = ska_sdp_config.Config(host=f"ska-sdp-etcd-client.{NAMESPACE}")
os.environ["SDP_CONFIG_HOST"] = f"ska-sdp-etcd-client.{NAMESPACE}"

## Having a processing block (and making it too)

Now all we need to do is create a processing block:

In [3]:
script = dict(kind='batch', name='notebook', version='3.14')
pb = ska_sdp_scripting.ProcessingBlock(None, script, None, cfg)

1|2022-09-13T07:15:43.218Z|DEBUG|MainThread|__init__|processing_block.py#57||Processing Block ID pb-nb-20220913-00001
1|2022-09-13T07:15:43.224Z|DEBUG|MainThread|__init__|processing_block.py#62||Processing Block ID pb-nb-20220913-00001
1|2022-09-13T07:15:43.249Z|INFO|MainThread|__init__|processing_block.py#84||Claimed processing block


In [4]:
for txn in cfg.txn():
    state = txn.get_processing_block_state(pb._pb_id)
    print(pb._pb_id, "\t", ", ".join(f"{key}: {val}" for key, val in state.items()) )

pb-nb-20220913-00001 	 last_updated: 2022-09-13 07:15:43, resources_available: False, status: STARTING


That's it! This is only remarkable because normally, `ska_sdp_scripting.ProcessingBlock` would claim an **existing** processing block. Here, two extra things happened:
1. The library automatically created an (SDP-internal) processing block matching the (bogus) processing script specification
2. It also immediately created the processing block state, preventing the processing controller from attempting to deploy said processing script

All three actions (create, claim, set status) are internally performed as a **transaction** so the processing controller never sees the incriminating in-between state!

## Getting ready for work

Just one more thing to get out of the way - before we are actually allowed to do anything, we first must first secure processing resources. Normally we'd now ask for an amount of computing resources (say 16 GPU nodes), and the processing controller would only grant them once they were available.
However, **this is not implemented yet**, so:

In [5]:
work_phase = pb.create_phase("Work", [])
work_phase.__enter__()

1|2022-09-13T07:15:43.263Z|INFO|MainThread|check_state|phase.py#105||Checking PB state
1|2022-09-13T07:15:43.279Z|INFO|MainThread|__enter__|phase.py#60||Setting status to WAITING
1|2022-09-13T07:15:43.291Z|INFO|MainThread|check_state|phase.py#105||Checking PB state
1|2022-09-13T07:15:43.314Z|INFO|MainThread|__enter__|phase.py#68||Waiting for resources to be available
1|2022-09-13T07:15:43.315Z|INFO|MainThread|check_state|phase.py#105||Checking PB state
1|2022-09-13T07:15:43.510Z|INFO|MainThread|check_state|phase.py#105||Checking PB state
1|2022-09-13T07:15:43.529Z|INFO|MainThread|__enter__|phase.py#74||Resources are available
1|2022-09-13T07:15:43.532Z|INFO|MainThread|check_state|phase.py#105||Checking PB state
1|2022-09-13T07:15:43.560Z|INFO|MainThread|__enter__|phase.py#87||Setting status to RUNNING
1|2022-09-13T07:15:43.560Z|INFO|MainThread|check_state|phase.py#105||Checking PB state


Asking for no resources (`[]`) means that the processing controller immediately grants our wish by advancing our state:

In [6]:
for txn in cfg.txn():
    state = txn.get_processing_block_state(pb._pb_id)
    print(pb._pb_id, "\t", ", ".join(f"{key}: {val}" for key, val in state.items()) )

pb-nb-20220913-00001 	 deployments: {}, last_updated: 2022-09-13 07:15:43, resources_available: True, status: RUNNING


It even states explicitly that our resources are "available". Anyway, our processing block is now officially **RUNNING**, so let's get to work...

## Revving up our engine

What can we do with this "work phase" we just entered? One option is to start a Dask execution engine:

In [7]:
helm_values = { "image": os.environ['JUPYTER_IMAGE'] }
deploy1 = work_phase.ee_deploy_dask("dask-1", 3, processing_ns=NAMESPACE+"-p", helm_values=helm_values)
while deploy1.client is None: time.sleep(0.1)
time.sleep(10) # Wait for workers to actually register

1|2022-09-13T07:15:43.599Z|INFO|Thread-8|_deploy|dask_deploy.py#81||Deploying Dask...
1|2022-09-13T07:15:43.600Z|INFO|Thread-8|_deploy|dask_deploy.py#83||proc-pb-nb-20220913-00001-dask-1
1|2022-09-13T07:15:43.633Z|INFO|Thread-8|_deploy|dask_deploy.py#112||Waiting for Dask...
1|2022-09-13T07:16:13.636Z|ERROR|Thread-8|_deploy|dask_deploy.py#122||Timed out trying to connect to tcp://proc-pb-nb-20220913-00001-dask-1-scheduler.dp-dppt-peter-p.svc.cluster.local:8786 after 30 s
1|2022-09-13T07:16:17.034Z|INFO|Thread-8|_deploy|dask_deploy.py#126||Connected to Dask


We can check that this worked:

In [8]:
deploy1.client

<Client: 'tcp://10.10.4.6:8786' processes=3 threads=84, memory=590.33 GiB>

This is especially associated with our processing block:

In [9]:
for txn in cfg.txn():
    state = txn.get_processing_block_state(pb._pb_id)
    print(pb._pb_id, "\t", ", ".join(f"{key}: {val}" for key, val in state.items()) )

pb-nb-20220913-00001 	 deployments: {'proc-pb-nb-20220913-00001-dask-1': 'RUNNING'}, last_updated: 2022-09-13 07:15:43, resources_available: True, status: RUNNING


## Making Dask sweat

We have quite a bit of memory to work with, so let's take an example from the documentation and scale it up:

In [10]:
import dask.array as da
x = da.random.random((100000, 100000), chunks=(10000, 10000))
x

dask.array<random_sample, shape=(100000, 100000), dtype=float64, chunksize=(10000, 10000), chunktype=numpy.ndarray>

In [11]:
y = x + x.T # Transposition! Nasty!
z = y[::2, 5000:].mean(axis=1)
z

dask.array<mean_agg-aggregate, shape=(50000,), dtype=float64, chunksize=(5000,), chunktype=numpy.ndarray>

In [12]:
%time z.compute()

CPU times: user 9.61 s, sys: 1.27 s, total: 10.9 s
Wall time: 2min 29s


array([1.00202096, 1.00093728, 0.9988442 , ..., 1.00015838, 1.00119531,
       1.0012998 ])

This takes quite a bit - mostly due to the network on the IRIS platform not being the fastest. However, the work has clearly been effectively pushed to the worker nodes

## Cleaning up

Let's "exit" the work phase!

In [13]:
deploy1.client.close()
work_phase.__exit__(None, None, None)

1|2022-09-13T07:18:53.494Z|INFO|MainThread|__exit__|phase.py#271||Deployments All Done


That's actually a lie - for some reason, batch deployments don't get cleaned up

In [14]:
for txn in cfg.txn():
    state = txn.get_processing_block_state(pb._pb_id)
    print(pb._pb_id, state )

pb-nb-20220913-00001 {'deployments': {'proc-pb-nb-20220913-00001-dask-1': 'RUNNING'}, 'last_updated': '2022-09-13 07:15:43', 'resources_available': True, 'status': 'FINISHED'}


distributed.batched - INFO - Batched Comm Closed <TCP (closed) Client->Scheduler local=tcp://10.10.212.88:54310 remote=tcp://proc-pb-nb-20220913-00001-dask-1-scheduler.dp-dppt-peter-p.svc.cluster.local:8786>
Traceback (most recent call last):
  File "/home/tango/.local/lib/python3.7/site-packages/distributed/batched.py", line 95, in _background_send
    payload, serializers=self.serializers, on_error="raise"
  File "/home/tango/.local/lib/python3.7/site-packages/tornado/gen.py", line 769, in run
    value = future.result()
  File "/home/tango/.local/lib/python3.7/site-packages/distributed/comm/tcp.py", line 248, in write
    raise CommClosedError()
distributed.comm.core.CommClosedError
distributed.client - ERROR - Failed to reconnect to scheduler after 30.00 seconds, closing client
distributed.client - ERROR - Failed to reconnect to scheduler after 30.00 seconds, closing client
distributed.client - ERROR - Failed to reconnect to scheduler after 30.00 seconds, closing client
distributed

But the processing block is marked as "FINISHED" at least, so we are all done...!